##Setup

In [74]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

##Import dataset

In [34]:
Dataframe_listings = pd.read_csv('/Listings.csv', encoding='latin1', parse_dates=['host_since'])

#pd.set_option('display.max_columns', None)

Dataframe_listings.info()

<ipython-input-34-b6665413f6c4>:1: DtypeWarning:

Columns (5,13) have mixed types. Specify dtype option on import or set low_memory=False.



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279712 entries, 0 to 279711
Data columns (total 33 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   listing_id                   279712 non-null  int64         
 1   name                         279537 non-null  object        
 2   host_id                      279712 non-null  int64         
 3   host_since                   279547 non-null  datetime64[ns]
 4   host_location                278872 non-null  object        
 5   host_response_time           150930 non-null  object        
 6   host_response_rate           150930 non-null  float64       
 7   host_acceptance_rate         166625 non-null  float64       
 8   host_is_superhost            279547 non-null  object        
 9   host_total_listings_count    279547 non-null  float64       
 10  host_has_profile_pic         279547 non-null  object        
 11  host_identity_verified    

## Filter Sur les annonces Parisiennes puis conserver que certaines colonnes

In [6]:
Dataframe_listings_filter = Dataframe_listings[Dataframe_listings['city'] == 'Paris']
Dataframe_listings_filter = Dataframe_listings_filter[['host_since', 'neighbourhood', 'city', 'accommodates', 'price']]

Dataframe_listings_filter

,host_since,neighbourhood,city,accommodates,price
0,2011-12-03,Buttes-Montmartre,Paris,2,53.0
1,2013-11-29,Buttes-Montmartre,Paris,2,120.0
2,2014-07-31,Elysee,Paris,2,89.0
3,2013-12-17,Vaugirard,Paris,2,58.0
4,2014-12-14,Passy,Paris,2,60.0
...,...,...,...,...,...
129590,2018-04-12,Popincourt,Paris,1,89.0
129591,2014-10-09,Vaugirard,Paris,4,190.0
129592,2014-10-09,Vaugirard,Paris,1,95.0
129634,2016-06-07,Elysee,Paris,3,310.0


## Vérifier la qualité des données

In [7]:
Dataframe_listings_filter.isnull().sum()
### host_since 33, neighbourhood	0, city	0, accommodates	0, price	0

,0
host_since,17
neighbourhood,0
city,0
accommodates,0
price,0


## Calculer le minimum, le maximum et la moyenne pour les champs numériques

In [8]:
Dataframe_listings_filter[['accommodates', 'price']].describe()



,accommodates,price
count,34132.000000,34132.000000
mean,3.211502,115.611332
std,1.663216,221.963207
min,0.000000,0.000000
25%,2.000000,60.000000
50%,3.000000,80.000000
75%,4.000000,120.000000
max,16.000000,12000.000000


## Regrouper les annonces parisiennes par « quartier » et calcule le prix moyen

In [30]:
paris_listing_quartier = Dataframe_listings_filter[['neighbourhood', 'price']]

paris_listing_quartier = paris_listing_quartier.groupby('neighbourhood').mean().sort_values('price', ascending= False)

paris_listing_quartier = paris_listing_quartier.reset_index()

paris_listing_quartier['price']= paris_listing_quartier['price'].round(decimals=1)

paris_listing_quartier

,neighbourhood,price
0,Elysee,201.9
1,Louvre,184.9
2,Passy,160.6
3,Luxembourg,155.9
4,Bourse,155.0
5,Palais-Bourbon,146.6
6,Temple,146.3
7,Hotel-de-Ville,144.3
8,Pantheon,125.2
9,Opera,118.9


## Regroupez par colonne « hébergement » et ajoutez le prix moyen pour chaque valeur de « hébergement »

In [28]:
paris_listings_accomodations = Dataframe_listings_filter[['accommodates', 'price']]

paris_listings_accomodations = paris_listings_accomodations.groupby('accommodates').mean().sort_values('price', ascending= False)

paris_listings_accomodations = paris_listings_accomodations.reset_index()

paris_listings_accomodations['price']= paris_listings_accomodations['price'].round(decimals=1)

paris_listings_accomodations


,accommodates,price
0,15,893.7
1,16,791.5
2,12,786.6
3,11,721.3
4,14,703.0
5,13,640.8
6,10,570.6
7,9,391.5
8,8,320.1
9,7,261.4


## Regrouper par année « host_since » et calculez le prix moyen et le nombre de lignes représentant le nombre de nouveaux hôtes.

In [11]:
Dataframe_listings_filter['year'] = Dataframe_listings_filter['host_since'].dt.year
paris_listings_over_time = Dataframe_listings_filter[['year', 'price']]
paris_listings_over_time = paris_listings_over_time.groupby('year').mean()
paris_listings_over_time

,price
year,
2008.0,125.000000
2009.0,239.722222
2010.0,131.361111
2011.0,111.831502
2012.0,110.288594
2013.0,107.051282
2014.0,100.676952
2015.0,111.863339
2016.0,122.006131


##Créez un graphique à barres horizontales du prix moyen par quartier à Paris

In [31]:
fig = px.bar(paris_listing_quartier,
             y='neighbourhood',
             x='price',
             text='price',
             color='neighbourhood',
             title="Prix moyen par quartier")
fig.show()

## Créer un grapique à barres horizontales du prix moyen par acceuil

In [29]:
fig = px.bar(
    paris_listings_accomodations,
    y ='price',
    x='accommodates',
    color= 'accommodates',
    text='price',
    title= 'Prix moyen par acceuil')

fig.show()

## Créez deux graphiques linéaires : un montrant le nombre de nouveaux hôtes au fil du temps et un montrant le prix moyen.

In [ ]:
# Calcul du nombre de hôtes par an
dataframe_filter_bis = Dataframe_listings[['host_id', "host_since"]]

dataframe_filter_bis['year']= dataframe_filter_bis['host_since'].dt.year

dataframe_filter_bis_count = dataframe_filter_bis.groupby('year').count().reset_index()

dataframe_filter_bis_count = dataframe_filter_bis_count[['year', 'host_id']]

# Calcul du prix moyen par an
dataframe_filter_ter = Dataframe_listings[['host_since', 'price']]

dataframe_filter_ter['year']= dataframe_filter_ter['host_since'].dt.year

dataframe_filter_ter_mean = dataframe_filter_ter.groupby('year').mean().reset_index()

dataframe_filter_ter_mean = dataframe_filter_ter_mean[['year', 'price']]

# Fusion des deux dataframes

dataframe_two_linear_graph = dataframe_filter_bis_count.merge(dataframe_filter_ter_mean, how ='left', on='year')

dataframe_two_linear_graph['price']= dataframe_two_linear_graph['price'].round(decimals =0)

dataframe_two_linear_graph

In [84]:
#Visualisation
fig = make_subplots(rows=1, cols=2, subplot_titles=("Nombre de clients par année", "Prix moyen par année"))

fig.add_trace(
    go.Scatter(x=dataframe_two_linear_graph['year'],
               y=dataframe_two_linear_graph['host_id']),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=dataframe_two_linear_graph['year'],
               y=dataframe_two_linear_graph['price']),

    row=1, col=2
)

fig.show()